Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**


**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques. Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display.**

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math
from __future__ import division
from scipy import stats

buffer_k_right = []
buffer_n_right = []
buffer_k_left = []
buffer_n_left = []


def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    plt.imshow(masked_image)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2, N = 1):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    global buffer_k_right
    global buffer_n_right 
    global buffer_k_left 
    global buffer_n_left
    
    k_right = []
    n_right = []
    k_left = []
    n_left = []
    
    center = img.shape[1] / 2 
    
    
    for line in lines:
        x1, y1, x2, y2 = line[0]  
        
        if(x2 - x1 != 0):
            
            slope = ((y2 - y1) / (x2 - x1))
            
            if abs(slope) < 1:
                
                if x1 >= center and x2 >= center and slope > 0:
        
                    k =  ((y2 - y1) / (x2 - x1))
                    k_right.append(k)
                    
                    n = y1 - k*x1
                    n_right.append(n)
                    
                elif x1 < center and x2 < center and slope < 0:

                    k =  ((y2 - y1) / (x2 - x1))
                    k_left.append(k)

                    n = y1 - k*x1
                    n_left.append(n)
                       
        else:
            continue
            
    k_avg_right = stats.trim_mean(k_right,  0.25)
    n_avg_right = stats.trim_mean(n_right,  0.25)  

    k_avg_left = stats.trim_mean(k_left,  0.25)
    n_avg_left = stats.trim_mean(n_left, 0.25)

    buffer_k_right.append(k_avg_right) 
    buffer_n_right.append(n_avg_right)
    buffer_k_left.append(k_avg_left) 
    buffer_n_left.append(n_avg_left)
  
    if len(buffer_k_right) > N:
        
        buffer_k_right = buffer_k_right[-N:]
        buffer_n_right = buffer_n_right[-N:]
        buffer_k_left = buffer_k_left[-N:]
        buffer_n_left = buffer_n_left[-N:]
        
        k_avg_right =  stats.trim_mean(buffer_k_right,  0.25)
        n_avg_right =  stats.trim_mean(buffer_n_right,  0.25)
        k_avg_left =  stats.trim_mean(buffer_k_left,  0.25)
        n_avg_left =  stats.trim_mean(buffer_n_left,  0.25)
    
        
    y1 = img.shape[0]
    y2 = img.shape[0] * (1 - 0.4)

    right_x1 = (y1 - n_avg_right) / k_avg_right
    right_x2 = (y2 - n_avg_right) / k_avg_right

    left_x1 = (y1 - n_avg_left) / k_avg_left
    left_x2 = (y2 - n_avg_left) / k_avg_left
    
    cv2.line(img, (int(right_x1), int(y1)), (int(right_x2), int(y2)), color, thickness)
    cv2.line(img, (int(left_x1), int(y1)), (int(left_x2), int(y2)), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, [255, 0, 0], 2, 5)
    return line_img
   

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)



## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
list_of_images = os.listdir("test_images/")
print(list_of_images)

# Build a Lane Finding Pipeline

#Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.
#Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

# Create output directory if it doesn't exist
test_images_output = ("test_images_output/")
os.makedirs(os.path.dirname(test_images_output), exist_ok=True)
print("Directory '%s' created" %test_images_output) 

# Loop through images in test_images folder
def test_image(image):
    
        # Transform the image in grayscale
        grayscale_result = grayscale(image)

        # Apply smoothing (gaussian blur)
        gaussian_blur_result = gaussian_blur(grayscale_result, 5)

        # Apply Canny edge detection
        canny_result = canny(gaussian_blur_result, 50, 150)                           

        # Select region of interest as traphesoid in front of the car and apply it on the image

        trap_bottom_width = 0.85  # width of bottom edge of trapezoid, expressed as percentage of image width
        trap_top_width = 0.07  # ditto for top edge of trapezoid
        trap_height = 0.4  # height of the trapezoid expressed as percentage of image height

        # image.shape[0] -> height ; image.shape[1] -> width image.shape[2] -> num_of_channels  

        vertices = np.array([[\
            ((image.shape[1] * (1 - trap_bottom_width)) // 2, image.shape[0]),\
            ((image.shape[1] * (1 - trap_top_width)) // 2, image.shape[0] - image.shape[0] * trap_height),\
            (image.shape[1] - (image.shape[1] * (1 - trap_top_width)) // 2, image.shape[0] - image.shape[0] * trap_height),\
            (image.shape[1] - (image.shape[1] * (1 - trap_bottom_width)) // 2, image.shape[0])]]\
            , dtype=np.int32)

        region_of_interest_result = region_of_interest(canny_result, vertices)

        # Apply Hough lines detector
        rho = 2 #distance resolution in pixels of the Hough grid
        theta = 1 * np.pi/180 #angular resolution in radians of the Hough grid
        threshold = 15 #minimum number of votes (intersections in Hough grid cell)
        min_line_len = 10 #minimum number of pixels making up a line
        max_line_gap = 20 # maximum gap in pixels between connectable line segments


        hough_lines_result  = hough_lines(region_of_interest_result, rho, theta, threshold, min_line_len, max_line_gap)

        weighted_img_result = weighted_img(hough_lines_result, image)

        return weighted_img_result
        

In [ ]:
def main_program():
    
    for img, i in zip(list_of_images, range(1,6)):
       
        path = r'test_images/'
        path = path + img

        # Reading an image in default mode 
        src = mpimg.imread(path) 

        result = test_image(src)

        plt.imshow(result)

        # Save the image
        name_str = 'test_images_output/' + 'output' + str(i) + '.jpg'
        print(name_str)
        optput_image = mpimg.imsave(name_str, result)
        print("Image successfully saved!")
        i = i + 1
        
main_program()
    

Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

solidWhiteRight.mp4

solidYellowLeft.mp4

Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.

If you get an error that looks like this:

NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()

Follow the instructions in the error message and check out this forum post for more troubleshooting tips across operating systems.


In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
trap_bottom_width = 0.7  # width of bottom edge of trapezoid, expressed as percentage of image width
trap_top_width = 0.1  # ditto for top edge of trapezoid
trap_height = 0.38  # height of the trapezoid expressed as percentage of image height

def trapezoid_roi(image, trap_bottom_width, trap_top_width, trap_height):

    src = np.float32([[(image.shape[1] * (1 - trap_bottom_width)) // 2, image.shape[0] - 50],\
                [(image.shape[1] * (1 - trap_top_width)) // 2, image.shape[0] - image.shape[0] * trap_height],\
                [image.shape[1] - (image.shape[1] * (1 - trap_top_width)) // 2, image.shape[0] - image.shape[0] * trap_height],\
                [image.shape[1] - (image.shape[1] * (1 - trap_bottom_width)) // 2, image.shape[0] - 50]])

    np.sort(src)

    dst = np.float32([[(image.shape[1] * (1 - trap_bottom_width)) // 2, image.shape[0]],\
                [(image.shape[1] * (1 - trap_bottom_width)) // 2, 0],\
                [image.shape[1] - (image.shape[1] * (1 - trap_bottom_width)) // 2, 0],\
                [image.shape[1] - (image.shape[1] * (1 - trap_bottom_width)) // 2, image.shape[0]]])
    
    return src, dst

def color_filter(image):
    
    white_hsv_low  = np.array([200,200,200])
    white_hsv_high = np.array([255,255,255])
    
    mask1 = cv2.inRange(image, white_hsv_low, white_hsv_high)
    
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    yellow_hsv_low = np.array([90,72,100])
    yellow_hsv_high = np.array([110,255,255])
    
    mask2 = cv2.inRange(image_hsv, yellow_hsv_low, yellow_hsv_high)

    mask = cv2.bitwise_or(mask1, mask2)
    
    white_img = cv2.bitwise_and(image,image, mask = mask1)
    yellow_img = cv2.bitwise_and(image,image, mask = mask2)
    
    filtered_image =  cv2.addWeighted(white_img, 1., yellow_img, 1., 0.)

    return filtered_image
    
def perspective_transform(image):

    filtered_img = color_filter(image)
    
    gray = cv2.cvtColor(filtered_img, cv2.COLOR_RGB2GRAY)

    src, dst = trapezoid_roi(image, trap_bottom_width, trap_top_width, trap_height)
    
    M = cv2.getPerspectiveTransform(src,dst)
    
    binary_threshold = np.zeros_like(gray)
    binary_threshold[(gray > 0)] = 1

    binary_warped = cv2.warpPerspective(binary_threshold,M,(image.shape[1],image.shape[0]))

    return binary_warped, binary_threshold

def get_histogram(image):
    
    histogram = np.sum(image[image.shape[0]//2:,:], axis=0)
    return histogram

def sliding_window(binary_warped):
    
    # Number of windows 
    N_windows = 9
    
    # Set window width and height
    window_width = 80 
    window_height = np.int(binary_warped.shape[0]/N_windows)
    
    # Create an output image to draw on and visualize the result
    output_image = (np.dstack((binary_warped, binary_warped, binary_warped))*255).astype('uint8')
    
    # Set minimum number of pixels found to recenter window
    min_pix = 50
    
    histogram = get_histogram(binary_warped)

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    left_x_base = np.argmax(histogram[:midpoint], axis = 0)
    right_x_base = np.argmax(histogram[midpoint:], axis = 0) + midpoint
    
    non_zero = binary_warped.nonzero()
    non_zero_y = np.array(non_zero[0])
    non_zero_x = np.array(non_zero[1])
    
    left_x_current = left_x_base
    right_x_current = right_x_base
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(N_windows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = int(binary_warped.shape[0] - (window + 1) * window_height)
        win_y_high = int(binary_warped.shape[0] - window * window_height)
        win_x_left_low = left_x_current - window_width
        win_x_left_high = left_x_current + window_width
        win_x_right_low = right_x_current - window_width
        win_x_right_high = right_x_current + window_width
        
        output_image  = cv2.rectangle(output_image,(win_x_left_low,win_y_low),(win_x_left_high,win_y_high),(0,255,0), 2)
        output_image = cv2.rectangle(output_image,(win_x_right_low,win_y_low),(win_x_right_high,win_y_high),(0,255,0), 2)

        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((non_zero_y >= win_y_low) & (non_zero_y < win_y_high) & (non_zero_x >= win_x_left_low) & (non_zero_x < win_x_left_high)).nonzero()[0]
        good_right_inds = ((non_zero_y >= win_y_low) & (non_zero_y < win_y_high) & (non_zero_x >= win_x_right_low) & (non_zero_x < win_x_right_high)).nonzero()[0]
    
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > min_pix:
            left_x_current = np.int(np.mean(non_zero_x[good_left_inds]))
        if len(good_right_inds) > min_pix:
            right_x_current = np.int(np.mean(non_zero_x[good_right_inds]))
            
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    # Extract left and right line pixel positions
    left_x = non_zero_x[left_lane_inds]
    left_y = non_zero_y[left_lane_inds]
    right_x = non_zero_x[right_lane_inds]
    right_y = non_zero_y[right_lane_inds]
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(left_y, left_x, 2)
    right_fit = np.polyfit(right_y, right_x, 2)
    
    ploty = np.linspace(0, output_image.shape[0]-1, output_image.shape[0])
    
    left_fit_x = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
    right_fit_x = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]
    
    return output_image, left_fit_x, right_fit_x, ploty

def inverse_perspective(binary_warped, original):

    out_warped, left_fit_x, right_fit_x, ploty = sliding_window(binary_warped)

    src, dst = trapezoid_roi(out_warped, trap_bottom_width, trap_top_width, trap_height)
    
    Minv = cv2.getPerspectiveTransform(dst,src)

    output_image = cv2.warpPerspective(out_warped,Minv,(out_warped.shape[1],out_warped.shape[0]))
    
    color_warp = np.zeros((out_warped.shape[0],out_warped.shape[1], out_warped.shape[2]), dtype='uint8')  # NOTE: Hard-coded image dimensions
    
    pts_left = np.array([np.transpose(np.vstack([left_fit_x, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fit_x, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (original.shape[1], original.shape[0]))
    
    # Combine the result with the original image
    result = cv2.addWeighted(original, 1, newwarp, 0.3, 0)
    
    return result



In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    
    result = test_image(image)
    
    return result

def process_image_advanced(image):

    binary_warped, binary_threshold = perspective_transform(image)
    
    output = inverse_perspective(binary_warped, image)
    
    return output
    

In [ ]:
white_output = 'test_videos_output/challenge_out1.mp4'
os.makedirs(os.path.dirname('test_videos_output/'), exist_ok=True)
print("Directory '%s' created" %test_images_output) 

## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("test_videos/challenge.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/challenge.mp4")
white_clip = clip1.fl_image(process_image_advanced) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))